# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [18]:
# imports
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import requests
import ollama

In [7]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [8]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [9]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [10]:
system_prompt = 'You are a personal assistant. You will be given a technical question and you have to answer it'

In [17]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

def get_gpt_response(user_prompt):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ],
        stream=True
    )

    response=''
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

get_gpt_response(question)

This code uses a generator expression combined with the `yield from` statement in Python, and it performs the following actions:

1. **Set Comprehension**: `{book.get("author") for book in books if book.get("author")}` creates a set of authors from a collection called `books`. Each `book` in `books` is expected to be a dictionary where the author's name can be retrieved by calling `book.get("author")`. The conditional clause `if book.get("author")` ensures that only books with a truthy value for the author (i.e., non-empty and not `None`) are included in the resulting set.

2. **Elimination of Duplicates**: Since this is a set comprehension, it automatically eliminates duplicate authors. If multiple books have the same author, that author will only appear once in the resulting set.

3. **Yield from**: The `yield from` statement is used within a generator function to yield all values from the given iterable (in this case, the set of authors). When this function is called, it will produce each author one at a time, effectively allowing the function to be iterated over.

In summary, this code iterates through a list of `books`, collects the unique authors of those books, and yields each author one by one. It is useful in scenarios where you want to process or generate a sequence of authors from a list of books without creating an intermediate list or a full set at once, thus maintaining efficiency in memory usage.

In [27]:
# Get Llama 3.2 to answer
def get_ollama_response(user_prompt):
    response = ollama.chat(model=MODEL_LLAMA, messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]
    )
    reply = response['message']['content']
    display(Markdown(reply))
    
get_ollama_response(question)


**Code Explanation**

The provided code snippet is written in Python and utilizes the `yield from` statement, which is a part of Python 3.3 onwards.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This line of code generates an iterator that yields the values from a dictionary comprehension. Here's a breakdown:

1. `{... for book in books ...}`: This is a dictionary comprehension, which creates a new dictionary (in this case) with the results of the expression inside the curly braces.

2. `if book.get("author")`: This filters the input data to only include dictionaries (`book`) where `"author"` exists as a key and its value is truthy (non-empty string, non-zero number, etc.). In Python 3, an empty string, zero, None, and False are considered falsy values.

3. `yield from ...`: The `yield from` statement is used to delegate a subexpression to a generator. When this expression is evaluated, it returns the results of that expression, which can be used as if they were generated by the outer function's generator.

So, in essence, this line of code generates an iterator over the authors of books where the author information exists.

**Why**

The `yield from` statement was introduced to improve the flexibility and readability of generators. By delegating a subexpression to another generator, it allows you to build complex iterators by combining simpler ones.

In this specific case, using `yield from` makes the code more readable because instead of nesting multiple levels of `for` loops or recursive function calls, we can use a dictionary comprehension and then delegate that result to another generator. This improves maintainability and reduces the chance of mistakes.

For example, without `yield from`, you might write something like this:

```python
def get_authors(books):
    authors = []
    for book in books:
        if book.get("author"):
            authors.append(book["author"])
    yield from authors
```

The code with `yield from` is more concise and easier to understand, making it a better choice when building complex iterators.